In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.13.1
11.7


c:\Users\PC\.conda\envs\Jupyter_Notebook\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch_geometric.data import Data
# edge index가 source와 dest로 주어질때
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data

Data(x=[3, 1], edge_index=[2, 4])

In [3]:
#edge index가 각각의 edge가 하나의 리스트로 주어질때
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

Data(x=[3, 1], edge_index=[2, 4])

# Bipartite_sage example
https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/bipartite_sage.py

In [4]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import Embedding, Linear

import torch_geometric.transforms as T
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import SAGEConv
from torch_geometric.nn.conv.gcn_conv import gcn_norm

In [5]:
%pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
path = osp.join(osp.dirname('C:/Users/PC/Desktop/MGCCF/PyG_tut.ipynb'), '../../data/MovieLens')
dataset = MovieLens(path, model_name='all-MiniLM-L6-v2')

In [7]:
data = dataset[0]
dataset[0]

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836]
  }
)

In [8]:
# re-mapping
data['user'].x = torch.arange(0,data['user'].num_nodes)
data['movie'].num_nodes = len(data['movie']['x'])
data['movie'].x = torch.arange(0,data['movie'].num_nodes)


In [9]:
data['user','movie'].edge_label

tensor([4, 4, 4,  ..., 5, 5, 3])

In [10]:
data['user','movie'].edge_label = data['user','movie'].edge_label.float()
data['user','movie'].edge_label

tensor([4., 4., 4.,  ..., 5., 5., 3.])

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
device

device(type='cuda')

In [12]:
data = T.ToUndirected()(data)

In [13]:
data

HeteroData(
  movie={
    x=[9742],
    num_nodes=9742
  },
  user={
    num_nodes=610,
    x=[610]
  },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836]
  },
  (movie, rev_rates, user)={
    edge_index=[2, 100836],
    edge_label=[100836]
  }
)

In [14]:
del data['movie', 'rev_rates', 'user'].edge_label

In [15]:
data

HeteroData(
  movie={
    x=[9742],
    num_nodes=9742
  },
  user={
    num_nodes=610,
    x=[610]
  },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836]
  },
  (movie, rev_rates, user)={ edge_index=[2, 100836] }
)

In [16]:
print("Computing data splits...")
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0, # 레이팅으로 하는 것의 경우 이것을 0.0 이 아닌 다른 값으로 잡으면 오류 발생
    add_negative_train_samples=False,
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')]
)(data)
print("Split Done")

Computing data splits...
Split Done


In [17]:
metapath = [('movie', 'rev_rates', 'user'), ('user', 'rates', 'movie')]
train_data = T.AddMetaPaths(metapaths=[metapath])(train_data)

In [18]:
_, edge_weight = gcn_norm(
    train_data['movie', 'movie'].edge_index,
    num_nodes=train_data['movie'].num_nodes,
    add_self_loops=False
)

In [19]:
edge_index = train_data['movie', 'movie'].edge_index[:, edge_weight > 0.002]

train_data['movie', 'metapath_0', 'movie'].edge_index = edge_index
val_data['movie', 'metapath_0', 'movie'].edge_index = edge_index
test_data['movie', 'metapath_0', 'movie'].edge_index = edge_index

In [20]:
train_data

HeteroData(
  metapath_dict={ (movie, metapath_0, movie)=[2] },
  movie={
    x=[9742],
    num_nodes=9742
  },
  user={
    num_nodes=610,
    x=[610]
  },
  (user, rates, movie)={
    edge_index=[2, 80670],
    edge_label=[80670],
    edge_label_index=[2, 80670]
  },
  (movie, rev_rates, user)={ edge_index=[2, 80670] },
  (movie, metapath_0, movie)={ edge_index=[2, 79981] }
)

In [21]:
class MovieGNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = SAGEConv(-1, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        return self.lin(x)

In [22]:
class UserGNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), hidden_channels)
        self.conv3 = SAGEConv((-1, -1), hidden_channels)
        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        movie_x = self.conv1(
            x_dict['movie'],
            edge_index_dict[('movie', 'metapath_0', 'movie')],
        ).relu()

        user_x = self.conv2(
            (x_dict['movie'], x_dict['user']),
            edge_index_dict[('movie', 'rev_rates', 'user')],
        ).relu()

        user_x = self.conv3(
            (movie_x, user_x),
            edge_index_dict[('movie', 'rev_rates', 'user')],
        ).relu()

        return self.lin(user_x)

In [23]:
class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_src, z_dst, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_src[row], z_dst[col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

In [24]:
class Model(torch.nn.Module):
    def __init__(self, num_users, num_movies, hidden_channels, out_channels):
        super().__init__()
        self.user_emb = Embedding(num_users, hidden_channels)
        self.movie_emb = Embedding(num_movies, hidden_channels)
        self.user_encoder = UserGNNEncoder(hidden_channels, out_channels)
        self.movie_encoder = MovieGNNEncoder(hidden_channels, out_channels)
        self.decoder = EdgeDecoder(out_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = {}
        x_dict['user'] = self.user_emb(x_dict['user'])
        x_dict['movie'] = self.movie_emb(x_dict['movie'])
        z_dict['user'] = self.user_encoder(x_dict, edge_index_dict)
        z_dict['movie'] = self.movie_encoder(
            x_dict['movie'],
            edge_index_dict[('movie', 'metapath_0', 'movie')],
        )
        return self.decoder(z_dict['user'], z_dict['movie'], edge_label_index)

In [25]:
model = Model(data['user'].num_nodes, data['movie'].num_nodes, hidden_channels=64, out_channels=64)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(
        train_data.x_dict,
        train_data.edge_index_dict,
        train_data['user', 'movie'].edge_label_index,
    )
    loss = F.mse_loss(out, train_data['user', 'movie'].edge_label)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    out = model(
        data.x_dict,
        data.edge_index_dict,
        data['user', 'movie'].edge_label_index,
    ).clamp(min=0, max=5)
    rmse = F.mse_loss(out, data['user', 'movie'].edge_label).sqrt()
    return float(rmse)

@torch.no_grad()
def predict(data):
    model.eval()
    out = model(
        data.x_dict,
        data.edge_index_dict,
        data['user', 'movie'].edge_label_index,
    ).clamp(min=0, max=5)
    
    return out


for epoch in range(1, 701):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    print(f'Epoch: {epoch:04d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

In [35]:
predict(test_data)

tensor([4.0233, 3.9060, 2.6514,  ..., 3.5669, 2.9768, 2.3004], device='cuda:0')